# Visualizing Camp Fire in California

In [1]:
import ee
import folium

If you're using this notebook for the first time you need to Authenticate the Earth Engine library with your google Earth Engine account. If you don't have an Earth Engine account then sign up from [here.](https://earthengine.google.com/signup/)

In [ ]:
ee.Authenticate()

In [2]:
ee.Initialize()

Also if you've never used Earth Engine's Python API [here's](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard) a sweet introduction from the documentation to get started.

I'd like to visualize California before camp fire, and it's devastating effect during & after the fire. So I'll be choosing this latitude and longitude as my point of interest.

In [3]:
lat =  39.444012
lon = -121.833619

In [4]:
my_map = folium.Map(location=[lat,lon])
my_map

In [5]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        name=name,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        overlay=True,
        control=True
    ).add_to(self)


folium.Map.add_ee_layer = add_ee_layer

In [6]:
landsat_sequence = [0,2,5]

In [7]:
poi = ee.Geometry.Point(lon,lat)

In [8]:
start_date = '2018-10-01'
end_date = '2019-01-31'

In [9]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(poi).filterDate(start_date, end_date)

In [10]:
landsat_list = landsat.toList(landsat.size());

Now I'll be defining a region of interest with a buffer zone of 20 km

In [11]:
roi = poi.buffer(20000) # meters

In [12]:
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [13]:
# Now lets Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

In [14]:
my_map.save('index.html')